In [1]:
from __future__ import print_function
import os
import numpy as np
import tensorflow as tf

import Tools
# from Tools import accuracy

In [6]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [37]:
def NNetmodel(
    num_output_unit,
    num_hid1_unit=500, 
    num_hid2_unit=100,
    momentum=0.9,
    learning_rate=0.1
    ):
    seed = 128
    rng = np.random.RandomState(seed)
    reset_graph()
    
    print (num_output_unit)
    print (num_hid1_unit)
    print (num_hid2_unit)
    print (momentum)
    print (learning_rate)
    
    x = tf.placeholder(tf.float32, shape=[None,None], name='input_placeholder')
    y = tf.placeholder(tf.float32, shape=[None,None], name='output_placeholder')
    num_input_unit = tf.shape(x)[1]   # x is of shape batch_size * no_of_features
    
    weights = {
        'input_to_hid1_wghts' : tf.Variable(tf.random_normal([1024,num_hid1_unit], seed=seed)),
        'hid1_to_output_wghts' : tf.Variable(tf.random_normal([num_hid1_unit,num_output_unit], seed=seed))
    }
    
    biases = {
        'hid1_bias': tf.Variable(tf.zeros([num_hid1_unit])),
        'output_bias' : tf.Variable(tf.zeros([num_output_unit]))
    }
    
    
    # Forward Propagate:
    input_to_hid1 = tf.matmul(x, weights['input_to_hid1_wghts']) + biases['hid1_bias']
    hid1_state = tf.sigmoid(input_to_hid1)
    
    hid1_to_output = tf.matmul(hid1_state, weights['hid1_to_output_wghts']) + biases['output_bias']
    output_state = tf.nn.softmax(hid1_to_output)
    
    # Error Derivative
    err_deriv = tf.sub(output_state, y, name=None)
    
    loss_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hid1_to_output, y))

    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_CE)
    
    optimizer = tf.train.MomentumOptimizer(learning_rate, 
                                            momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(loss_CE)
    
    return dict(
        x = x,
        y = y,
        loss = loss_CE,
        optimizer = optimizer,
        prediction = output_state,
        num_input_unit = num_input_unit
#         saver = tf.train.Saver()
    )

In [38]:
# Train the Network
def trainNetwork(graph_dict):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        training_loss = 0
        epochs = 10
        for epoch in np.arange(epochs):
            for trnData, trnLabels in batch_file_iterator(path_to_batches, batch_filename_arr):
                num_features = trnData.shape[1]*trnData.shape[2]
                trnDataRshp, trnLabelsRshp = Tools.reshape_data(trnData, 
                                                          trnLabels, 
                                                          num_features=num_features,
                                                          num_labels = 10
                                                         )
                                    
#                 print (trnData)
#                 print (trnLabels)#trnData.shape[2])
                feed_dict = {
                    graph_dict['x']:trnDataRshp, 
                    graph_dict['y']:trnLabelsRshp
                }

                loss_, opt_, pred_, aa= sess.run([graph_dict['loss'],
                                              graph_dict['optimizer'],
                                              graph_dict['prediction'],
                                              graph_dict['num_input_unit']], 
                                              feed_dict=feed_dict)
                print (aa)
                training_loss += loss_
                acc = Tools.accuracy(prediction=pred_, labels=trnLabelsRshp, labels_one_hot='Yes')
                print (training_loss)
                print (acc)
          

In [39]:


parent_dir = parent_dir = '/Users/sam/All-Program/App-DataSet/Kaggle-Challenges/CIFAR-10/'
fetaureType = 'featureSTD' 
path_to_batches = parent_dir + fetaureType + '/batchPath'
batch_filename_arr = np.array(os.listdir(path_to_batches))
np.random.shuffle(batch_filename_arr)
print (batch_filename_arr)

graph_dict = NNetmodel(num_output_unit=10,
                       num_hid1_unit=100, 
                       momentum=0.9,
                       learning_rate=0.1
                      )
# print (graph_dict)
trainNetwork(graph_dict)#, path_save_model=path_save_model)
# sys.stdout.close()

['batch9.pickle' 'batch4.pickle' 'batch0.pickle' 'batch6.pickle'
 'batch7.pickle' 'batch5.pickle' 'batch1.pickle' 'batch3.pickle'
 'batch8.pickle' 'batch2.pickle']
10
100
100
0.9
0.1
1024
10.466843605
7.25
1024
19.9737930298
7.3
1024
28.5198564529
7.625
1024
36.1298980713
8.65
1024
43.2476010323
7.925
1024
49.7398619652
8.975
1024
55.8409657478
10.25
1024
61.6292657852
10.725
1024
67.2206835747
10.975
1024
72.6140298843
12.075
1024
77.9640345573
12.175
1024
83.1613073349
14.125
1024
88.2264742851
13.8
1024
93.2313694954
13.925
1024
98.2056474686
14.6
1024
103.043973446
13.75
1024
107.725228786
15.35
1024
112.399525642
15.5
1024
117.066029549
15.375
1024
121.570713043
15.675
1024
126.013179779
15.825
1024
130.352700233
16.775
1024
134.690052986
16.2
1024
139.024157524
16.375
1024
143.375281334
16.775
1024
147.662895203
15.275
1024
151.813127995
17.325
1024
155.96649313
17.025
1024
160.078063965
17.225
1024
164.06761837
16.55
1024
168.03015995
17.475
1024
171.931862354
18.275
1024
175.81

In [25]:
no_output_unit = 10
no_of_labels = 10
no_inp_unit = 1024
no_hid_unit = 100
learning_rate=0.1
momentum = 0.9

graph = tf.Graph()
with graph.as_default():
    seed = 128
    rng = np.random.RandomState(seed)
    tf_training_dataset = tf.placeholder(tf.float32, shape=(None, no_inp_unit))
    tf_training_labels = tf.placeholder(tf.float32, shape=(None, no_of_labels))

    weights = {
        'input_to_hid_wghts': tf.Variable(tf.random_normal([no_inp_unit, no_hid_unit], seed=seed)),
        'hid_to_output_wghts': tf.Variable(tf.random_normal([no_hid_unit, no_output_unit], seed=seed))
    }

    biases = {
        'hid_bias' : tf.Variable(tf.zeros([no_hid_unit])),
        'output_bias' : tf.Variable(tf.zeros([no_output_unit]))
    }

    ###### Forward Propagate ######
    # Hidden Layer
    input_to_hid_layer = tf.matmul(tf_training_dataset, weights['input_to_hid_wghts']) + biases['hid_bias']
    hid_layer_state = tf.sigmoid(input_to_hid_layer, name=None)

    # Output Layer
    hid_to_output_layer = tf.matmul(hid_layer_state, weights['hid_to_output_wghts']) + biases['output_bias']
    output_layer_state = tf.nn.softmax(hid_to_output_layer, name=None)
    error_derivative = tf.sub(output_layer_state, tf_training_labels, name=None)  # -(y - y_hat) = (y_hat - y)
    # The above three lines of code can also be combined into one line as below.
    loss_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hid_to_output_layer, tf_training_labels))

    
    optimizer = tf.train.MomentumOptimizer(learning_rate, 
                                            momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(loss_CE)
    # Training Prediction
    training_prediction = output_layer_state


In [26]:

epochs = 1
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("All variable Initialized successfully")
    for epoch in range(epochs):
        # Retreive mini-batches
        for trnData, trnLabels in batch_file_iterator(path_to_batches, batch_filename_arr):
            num_features = trnData.shape[1]*trnData.shape[2]
            trnDataRshp, trnLabelsRshp = reshape_data(trnData, 
                                                          trnLabels, 
                                                          num_features=num_features,
                                                          num_labels = 10
                                                         )
                                    
            print (trnDataRshp.shape)
            print (trnLabelsRshp.shape)
#                 print (trnData.shape[1], trnData.shape[2])
#             feed_dict = {
#                     graph_dict['x']:trnDataRshp, 
#                     graph_dict['y']:trnLabelsRshp
#                 }
    

                
            feed_dict = {tf_training_dataset : trnDataRshp, tf_training_labels : trnLabelsRshp}
            _, l, predictions = session.run([optimizer, loss_CE, training_prediction], feed_dict=feed_dict)
#             print (predictions)
        
#         if (epoch % 2 == 0) or epoch == epochs-1:
            print("Minibatch loss at epoch %d: %f" % (epoch, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, trnLabelsRshp))
#             print("Validation accuracy: %.1f%%" % accuracy(crossvalid_prediction.eval(), crossvalid_labels_))
#     print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels_))

All variable Initialized successfully
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 10.507124
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 9.620941
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 8.508722
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)


/Users/sam/App-Setup/CondaENV/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Minibatch loss at epoch 0: 7.794838
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 6.984330
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 6.502948
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 6.112256
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 5.684135
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 5.508185
Minibatch accuracy: 0.0%
(4000, 1024)
(4000, 10)
Minibatch loss at epoch 0: 5.472131
Minibatch accuracy: 0.0%
